In [3]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# 1. Load Data
file_path = '../../JBI100 Data (2025-2026)/Hospital Beds Management/patients.csv'
df_pat = pd.read_csv(file_path)

# 2. Date Conversion
df_pat['arrival_date'] = pd.to_datetime(df_pat['arrival_date'])
df_pat['departure_date'] = pd.to_datetime(df_pat['departure_date'])

# 3. Feature Engineering: Length of Stay (LoS)
# We add 1 day if arrival == departure to avoid 0-day bugs in log-scales later, 
# or treat them as same-day 0. Let's keep raw difference for now.
df_pat['length_of_stay'] = (df_pat['departure_date'] - df_pat['arrival_date']).dt.days

# 4. Feature Engineering: Temporal Granularity
df_pat['arrival_month'] = df_pat['arrival_date'].dt.month_name()
df_pat['arrival_day_of_week'] = df_pat['arrival_date'].dt.day_name()
df_pat['week_number'] = df_pat['arrival_date'].dt.isocalendar().week

# 5. Feature Engineering: Bins for Categorical Analysis (Crucial for Parallel Sets/Categories)
# Age Groups
df_pat['age_group'] = pd.cut(df_pat['age'], bins=[0, 18, 40, 65, 100], labels=['Child', 'Young Adult', 'Adult', 'Senior'])
# Satisfaction Levels
df_pat['sat_level'] = pd.cut(df_pat['satisfaction'], bins=[0, 40, 70, 100], labels=['Low', 'Medium', 'High'])

print("Data Loaded & Engineered.")
print(f"Total Patients: {len(df_pat)}")
display(df_pat.head())

Data Loaded & Engineered.
Total Patients: 1000


,patient_id,name,age,arrival_date,departure_date,service,satisfaction,length_of_stay,arrival_month,arrival_day_of_week,week_number,age_group,sat_level
0,PAT-09484753,Richard Rodriguez,24,2025-03-16,2025-03-22,surgery,61,6,March,Sunday,11,Young Adult,Medium
1,PAT-f0644084,Shannon Walker,6,2025-12-13,2025-12-14,surgery,83,1,December,Saturday,50,Child,High
2,PAT-ac6162e4,Julia Torres,24,2025-06-29,2025-07-05,general_medicine,83,6,June,Sunday,26,Young Adult,High
3,PAT-3dda2bb5,Crystal Johnson,32,2025-10-12,2025-10-23,emergency,81,11,October,Sunday,41,Young Adult,High
4,PAT-08591375,Garrett Lin,25,2025-02-18,2025-02-25,ICU,76,7,February,Tuesday,8,Young Adult,High


In [4]:
# Create a complex scatter plot
fig = px.scatter(
    df_pat, 
    x="length_of_stay", 
    y="satisfaction", 
    color="service", 
    trendline="ols", # Ordinary Least Squares to check correlation
    marginal_x="violin", # Show distribution of Stay duration on top
    marginal_y="box",    # Show distribution of Satisfaction on side
    hover_data=["name", "age", "age_group"],
    title="Impact of Length of Stay on Patient Satisfaction per Service",
    labels={"length_of_stay": "Days in Hospital", "satisfaction": "Satisfaction Score (0-100)"},
    opacity=0.6
)

fig.update_layout(height=700, template="plotly_white")
fig.show()

In [5]:
# We use the binned data created in Cell 1
fig = px.parallel_categories(
    df_pat, 
    dimensions=['service', 'age_group', 'sat_level'],
    color="satisfaction", # Color by the raw score for gradient effect
    color_continuous_scale=px.colors.diverging.Tealrose,
    title="Patient Flow: Service -> Age Group -> Satisfaction Outcome",
    labels={'service':'Department', 'age_group':'Age Cohort', 'sat_level':'Satisfaction'}
)

fig.update_layout(height=600)
fig.show()

In [7]:
# Group by Service and Week Number to get admission counts
heatmap_data = df_pat.groupby(['service', 'week_number']).size().reset_index(name='count')

fig = px.density_heatmap(
    heatmap_data, 
    x="week_number", 
    y="service", 
    z="count", 
    nbinsx=52, # One bin per week
    color_continuous_scale="Viridis",
    title="Service Load Heatmap: Admissions per Week",
    labels={"week_number": "Week of Year", "service": "Department"}
)

fig.update_layout(xaxis=dict(tickmode='linear', tick0=1, dtick=5))
fig.show()

In [ ]:
# 1. Determine dynamic thresholds based on the data
high_los_cutoff = df_pat['length_of_stay'].quantile(0.90) # Top 10% longest stays
low_sat_cutoff = df_pat['satisfaction'].quantile(0.10)    # Bottom 10% satisfaction scores

print(f"Filtering for Stays > {high_los_cutoff:.1f} days AND Satisfaction < {low_sat_cutoff:.1f}")

# 2. Filter data
outliers = df_pat[
    (df_pat['length_of_stay'] >= high_los_cutoff) & 
    (df_pat['satisfaction'] <= low_sat_cutoff)
]

print(f"Found {len(outliers)} critical cases.")

# 3. Plot (Only if data exists)
if not outliers.empty:
    fig = px.scatter(
        outliers,
        x="age",
        y="length_of_stay",
        size="length_of_stay", 
        color="service",
        hover_data=["satisfaction", "name"],
        text="name", 
        title=f"Critical Cases: Long Stay (> {int(high_los_cutoff)} days) & Low Satisfaction (< {int(low_sat_cutoff)})",
        labels={"age": "Patient Age", "length_of_stay": "Days in Hospital"}
    )
    
    # Clean up the text labels so they don't overlap too much
    fig.update_traces(textposition='top center')
    fig.update_layout(height=600)
    fig.show()
else:
    print("No outliers found! Try widening the quantiles (e.g., 0.80 for LoS and 0.25 for Satisfaction).")

Filtering for Stays > 13.0 days AND Satisfaction < 63.0
Found 14 critical cases.
